In [1]:
from gen_ai_hub.proxy.core.proxy_clients import set_proxy_version
from hana_ml import dataframe
from hana_ml.algorithms.pal.utility import DataSets

#use btp proxy llm
#

In [4]:
from gen_ai_hub.proxy.langchain import init_llm
from hana_ai.agents.hana_dataframe_agent import create_hana_dataframe_agent
from hana_ai.vectorstore.embedding_service import GenAIHubEmbeddings
llm = init_llm('gpt-4', temperature=0.0, max_tokens=512)


c:\Users\I308290\AppData\Local\miniconda3\Lib\site-packages\langchain_experimental\tools\__init__.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_experimental.tools.python.tool import PythonAstREPLTool, PythonREPLTool
c:\users\i308290\hana.ai\src\hana_ai\tools\code_template_tools.py:20: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be us

In [2]:
#connection to hana
#url, port, user = "hanagpu02.lenovo.wdf.sap.corp", 30015, "USER1"
url="hcp-ml-validate.hana-ml.c.ap-cn-1.cloud.sap"
user="MLAPITESTER"
passwd="Abcd12345"
port=30315
connection_context = dataframe.ConnectionContext(url, port, user, passwd)
hana_df, _, _, _ = DataSets.load_boston_housing_data(connection_context)
hana_df = hana_df.to_head("ID")

Table BOSTON_HOUSING_PRICES exists.


In [3]:
hana_df.collect()

,ID,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,BLACK,LSTAT,MEDV
0,339,0.03306,0.0,5.19,0,0.515,6.059,37.3,4.8122,5,224,20.2,396.14,8.51,20.6
1,340,0.05497,0.0,5.19,0,0.515,5.985,45.4,4.8122,5,224,20.2,396.90,9.74,19.0
2,341,0.06151,0.0,5.19,0,0.515,5.968,58.5,4.8122,5,224,20.2,396.90,9.29,18.7
3,342,0.01301,35.0,1.52,0,0.442,7.241,49.3,7.0379,1,284,15.5,394.74,5.49,32.7
4,343,0.02498,0.0,1.89,0,0.518,6.540,59.7,6.2669,1,422,15.9,389.96,8.65,16.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,337,0.03427,0.0,5.19,0,0.515,5.869,46.3,5.2311,5,224,20.2,396.90,9.80,19.5
502,167,2.01019,0.0,19.58,0,0.605,7.929,96.2,2.0459,5,403,14.7,369.30,3.70,50.0
503,338,0.03041,0.0,5.19,0,0.515,5.895,59.6,5.6150,5,224,20.2,394.81,10.56,18.5
504,168,1.80028,0.0,19.58,0,0.605,5.877,79.2,2.4259,5,403,14.7,227.61,12.14,23.8


<img src="dfagent.png" />

In [ ]:
from hana_ai.tools.toolkit import HANAMLToolkit
from hana_ai.vectorstore.hana_vector_engine import HANAMLinVectorEngine

toolkit = HANAMLToolkit(connection_context)
hana_vec = HANAMLinVectorEngine(connection_context, "hana_vec_hana_ml_python_knowledge")
hana_vec.create_knowledge()
toolkit.set_vectordb(hana_vec)


In [46]:
knowledge_df = hana_vec.get_knowledge()

In [ ]:
knowledge_df.head(5).collect()

In [ ]:
agent = create_hana_dataframe_agent(llm=llm, tools=toolkit.get_tools(), df=hana_df, verbose=True, handle_parsing_errors=True)

In [48]:
# query = """
# Question: create a dataset report.
# Code template: 
# from hana_ml.visualizers.unified_report import UnifiedReport
# UnifiedReport(<data_frame>).build(key=None).display()

# """

query = """
to use boston housing price table to create a dataset report.

"""

In [ ]:
agent.invoke(query)

In [ ]:
agent.run("create a dataset report.")

In [ ]:
agent.invoke("Create Automatic Regression model on this dataframe with max_eval_time_mins=10. Provide key is ID, background_size=100 and model_table_name='my_model' in the fit function and execute it. ")

In [ ]:
agent = create_hana_dataframe_agent(llm=llm, tools=toolkit.get_tools(), df=hana_df.deselect("MEDV").head(1), verbose=True)
agent.run("Do the predict on this dataframe with key 'ID' and model 'mymodel'. Save the predict result as the table name 'predcit_result'. Explain the predict result based on the reason code.")